In [40]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3


In [3]:
#Database   Creation and  initilization 
conn = sqlite3.connect('airport.db')
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS flights(
               id INTEGER PRIMARY KEY,
               name TEXT NOT NULL,
               travelling TEXT NOT NULL,
               status TEXT NOT NULL)""")
conn.commit()
conn.close


<function Connection.close()>

In [8]:
#Funcation to insert the data into database 
def insert_into_database(passenger):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute(
    "INSERT INTO flights (id,name,travelling,status) VALUES (?,?,?,?)",
    passenger
    )
    conn.commit()
    conn.close()


In [9]:
data_passengers=[
        (101, "Ali", "Lahore to Karachi", "Not Booked"),
        (102, "Ahmed", "Lahore to Karachi", "Booked"),
        (103, "Hassan", "Lahore to Karachi", "Not Booked")
    ]


In [10]:
for d in data_passengers:
    insert_into_database(d)

In [4]:
import sqlite3

def view_database():
    with sqlite3.connect('airport.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM flights")
        rows = cursor.fetchall()

    # connection auto-closed here

    for row in rows:
        print(row)

    return rows

In [47]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'NOT BOOKED')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'Not Booked')


[(101, 'Ali', 'Lahore to Karachi', 'NOT BOOKED'),
 (102, 'Ahmed', 'Lahore to Karachi', 'Booked'),
 (103, 'Hassan', 'Lahore to Karachi', 'Not Booked')]

In [61]:
def booked_flight(status, id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE id = ?", (status, id))
    conn.commit()

    cursor.execute("SELECT * FROM flights WHERE id = ?", (id,))
    row = cursor.fetchone()  # get the updated row

    conn.close()
    return row  # return the tuple

In [50]:
booked_flight('booked',101)

[(101, 'Ali', 'Lahore to Karachi', 'booked')]

In [51]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'booked')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'Not Booked')


[(101, 'Ali', 'Lahore to Karachi', 'booked'),
 (102, 'Ahmed', 'Lahore to Karachi', 'Booked'),
 (103, 'Hassan', 'Lahore to Karachi', 'Not Booked')]

In [62]:
def cancel_ticket(status, id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE id = ?", (status, id))
    conn.commit()

    cursor.execute("SELECT * FROM flights WHERE id = ?", (id,))
    row = cursor.fetchone()

    conn.close()
    return row

In [11]:
cancel_ticket('NOT Booked',101)

In [73]:
view_database()

(101, 'Ali', 'Lahore to Karachi', 'booked')
(102, 'Ahmed', 'Lahore to Karachi', 'Booked')
(103, 'Hassan', 'Lahore to Karachi', 'booked')


[(101, 'Ali', 'Lahore to Karachi', 'booked'),
 (102, 'Ahmed', 'Lahore to Karachi', 'Booked'),
 (103, 'Hassan', 'Lahore to Karachi', 'booked')]

In [41]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('GEMINI_API_KEY')


In [42]:
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key = OPENAI_API_KEY)

In [15]:
response = client.chat.completions.create(model = "gemini-2.5-flash",messages={'role':'user','content':'hellow'})

In [16]:
print(response.choices[0].message.content)

Hello! How can I help you today?


In [17]:
viewAll_data = {
    'name':'view_database',
    'description':'Call this function when user went see all of the database',
    'parameters':{
        'type':'object',
        "properties":{},
        'required':[]
    }
    
}

In [18]:
booked_tool = {
    "name":'booked_flight',
    "description":"User this tool when user went to booked the flight",
    'parameters':{
        'type':'object',
        "properties":{
            'status':{
                'type':'string',
                'description':'This parameter represent the  status in database they booked or not'
            },
            'id':{
             'type':'integer',
             'description':'This parameters is Primary key of user , using this we access the user data and update it'
            }
        },
        'required':['status','id']
    }
}

In [19]:
ticket_cancel_tool = {
    "name":'cancel_ticket',
    'description':'Used this function when user went to cancel their booked ticket',
    'parameters':{
        'type':'object',
        'properties':{
            'status':{
                'type':'string',
                'description':'The parameter required to update the user status form booked to NOT-Booked'
            },
            'id':{
                'type':'integer',
                'description': 'This is paramter is primary key , using this we acces the user data from database'

            },    
        },
        'required': ['status','id'],
    }
    
}

In [20]:
tool = [viewAll_data, booked_tool, ticket_cancel_tool]

In [21]:
tools = [
    {"type": "function", "function": viewAll_data},
    {"type": "function", "function": booked_tool},
    {"type": "function", "function": ticket_cancel_tool}
]

In [22]:
import json
print("Tools structure:")
print(json.dumps(tools, indent=2))

Tools structure:
[
  {
    "type": "function",
    "function": {
      "name": "view_database",
      "description": "Call this function when user went see all of the database",
      "parameters": {
        "type": "object",
        "properties": {},
        "required": []
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "booked_flight",
      "description": "User this tool when user went to booked the flight",
      "parameters": {
        "type": "object",
        "properties": {
          "status": {
            "type": "string",
            "description": "This parameter represent the  status in database they booked or not"
          },
          "id": {
            "type": "integer",
            "description": "This parameters is Primary key of user , using this we access the user data and update it"
          }
        },
        "required": [
          "status",
          "id"
        ]
      }
    }
  },
  {
    "type": "function",
    "function":

In [23]:
system_message = """
You are an intelligent Airport Assistant.

You help users with:
1. Booking a flight
2. Canceling a flight
3. Viewing booking information

Rules:
- If the user wants to book a flight, call the tool "booked_flight".
- If the user wants to cancel a flight, call the tool "cancel_ticket".
- If the user wants to see booking information, call the tool "view_database".
- Do NOT manually generate booking confirmation without calling the correct tool.
- Always extract the user ID from the message and pass it to the tool.
- Do not answer from memory. Always rely on the tool for database-related tasks.
"""

In [74]:
#function for tool handling
def handle_tool(content):
    content1 = content.tool_calls[0]
    response = {}

    if content1.function.name == 'view_database':
        rows = view_database()  # Your DB function
        # Convert each row to a readable string
        result_str = "\n".join([f"ID: {r[0]}, Name: {r[1]}, Travelling: {r[2]}, Status: {r[3]}" for r in rows])
        response = {
            'role': 'tool',
            'content': result_str,
            'tool_call_id': content1.id
        }

    elif content1.function.name == 'booked_flight':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments, str) else content1.function.arguments
        arg_status = args['status']
        arg_id = args['id']
        result = booked_flight(arg_status, arg_id)  # Your DB update function
        # Convert result to string
        result_str = f"ID: {result[0]}, Name: {result[1]}, Travelling: {result[2]}, Status: {result[3]}"
        response = {
            'role': 'tool',
            'content': result_str,
            'tool_call_id': content1.id
        }

    elif content1.function.name == 'cancel_ticket':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments, str) else content1.function.arguments
        args_status = args['status']
        args_id = args['id']
        result = cancel_ticket(args_status, args_id)  # Your DB update function
        # Convert result to string
        result_str = f"ID: {result[0]}, Name: {result[1]}, Travelling: {result[2]}, Status: {result[3]}"
        response = {
            'role': 'tool',
            'content': result_str,
            'tool_call_id': content1.id
        }

    return response

In [75]:
# Chat function to interact with Gemini LLM
def chat_with_ai(message, history):
    # Prepare message history
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]

    # First call to LLM
    first_response = client.chat.completions.create(
        model='gemini-2.5-flash',
        messages=messages,
        tools=tools,
        tool_choice='auto'
    )

    # If tool call happens
    if first_response.choices[0].finish_reason == 'tool_calls':
        content = first_response.choices[0].message
        tool_call = content.tool_calls[0]

        # Handle tool call
        result = handle_tool(content)

        # Append tool call and result back to messages
        messages.append({
            "role": "assistant",
            "tool_calls": content.tool_calls
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result['content']
        })

        # Second call to LLM after tool result
        second_response = client.chat.completions.create(
            model='gemini-2.5-flash',
            messages=messages,
            tools=tools,
            tool_choice='auto'
        )
        return second_response.choices[0].message.content

    # If no tool call, just return normal LLM reply
    return first_response.choices[0].message.content

In [71]:
import gradio as gr

In [72]:
gr.ChatInterface(fn=chat_with_ai,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
